In [ ]:
# File: notebooks/debug_contracts.ipynb

# Cell 1: Import thư viện
import json
import pandas as pd
import re
import os



In [ ]:

# Cell 2: Load raw text file
raw_file_path = "../store/temp/your_contract_raw.json"  # Update tên file của bạn

with open(raw_file_path, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

raw_text = raw_data.get("raw_text", "")
print(raw_text[:1000])  # In thử 1000 ký tự đầu tiên



In [ ]:
# Cell 3: Tách thử Block Room Rates
rates_block = re.search(r"B\u1ea2NG GI\u00c1 PH\u00d2NG(.*?)(Ch\u00ednh s\u00e1ch tr\u1ebb em|Ch\u00ednh s\u00e1ch hu\u1ef7 ph\u00f2ng|\Z)",
                        raw_text, re.DOTALL | re.IGNORECASE)

if rates_block:
    rates_text = rates_block.group(1)
    print(rates_text[:1000])  # In thử nội dung bảng giá phòng
else:
    print("Không tìm thấy bảng giá phòng!")



In [ ]:
# Cell 4: Parse các dòng giá phòng thử
room_rates = []

for line in rates_text.split("\n"):
    line = line.strip()
    match = re.search(r"(?P<room_type>.+?)\s*-\s*(?P<size>\d+)\s*m2\s*-\s*(?P<bed_type>.+?)\s*-\s*(?P<normal_price>[0-9\.]+)[/\\](?P<high_price>[0-9\.]+)",
                      line)
    if match:
        room_rates.append({
            "room_type": match.group("room_type").strip(),
            "size_sqm": int(match.group("size")),
            "bed_type": match.group("bed_type").strip(),
            "normal_season_rate": int(match.group("normal_price").replace(".", "")),
            "high_season_rate": int(match.group("high_price").replace(".", ""))
        })



In [ ]:
# Cell 5: Xem kết quả bằng pandas
if room_rates:
    df_rates = pd.DataFrame(room_rates)
    display(df_rates)
else:
    print("Không parse được dòng giá phòng nào!")



In [ ]:
# Cell 6: Lưu tạm DataFrame thành CSV (tuỳ chọn)
# df_rates.to_csv("../store/output/room_rates_debug.csv", index=False)